<img src="pic/icon_exercise_header_p2.png" width="100%"> <br>
# ICON Community Interface ComIn - Practical Exercise Notebooks  
# <br> Exercise P2: Masking (Non-)Prognostic Cells

---
* **Exercise P2**: In this exercise, we explore what prognostic cells are and learn how to define a mask for them using Python.  
* This exercise is an extension of <a href="P1_exercise.ipynb"><code>P1_exercise</code></a>. To proceed, ensure that you have the binary `icon` configured with the `--enable-comin` option and the ComIn Python adapter obtained in the **previous exercise**.


---

<figure style="float:right;">
<img src="pic/index_ordering_exercise_illustration01.png" width="700">
</figure>

**Introduction:<br/> The computational domain in ICON:**
* **Domain decomposition** is essential for achieving scalability in grid point models on modern parallel computers.
* Each model domain is divided and distributed across multiple **processing elements (PE)**.
* We do not need to perform numerical calculations on each cell: The rows of cells at the **lateral boundaries** have prescribed values, and we also do not need to calculate values in the so-called **halo region**, the region around a processor's domain where partial information from neighbouring processors is stored. Only the **inner cells (prognostic cells)** need to be calculated.
* The figure schematically shows the different parts of a computational domain, subdivided between two PEs: Each PE “owns” a subset of cell rows at the lateral boundary, the nudging zone, the inner cells (prognostic cells), and the halo region.

For a more detailed information, refer to Section 9.3 of the ICON tutorial (see the [link below](icon_exercise_programming.ipynb#Further-Reading-and-Resources)).

In the next exercise, our objective is to assign the variable we've created to `mpi_process_rank` only for *prognostic* cells. While in the Fortran code one must use loops for this purpose, in our Python plugin we rely on masked array operations instead of loops. 

A possibility to distinguish between prognostic cells and halo points is provided by the `cells.decomp_domain` data structure.  

<div class="alert alert-success">
  <b style="color:#2d4b9b;">Exercise:</b> 
  Extend your plugin script <a href="scripts/comin_plugin.py"><code><code style="color:#2d4b9b">scripts/comin_plugin.py</code></a>: Generate a mask to identify prognostic cells. Use this mask to assign the variable <code>comin_process_id</code> to <code>mpi_process_rank</code> only for prognostic cells and set it to <code>-1</code> elsewhere.
   When you have finished modifying the plugin, run the ICON model again as you did in the previous part.
</div>

**Hints:**  

* This exercise involves accessing the descriptive data structures in the ComIn plugins, which are crucial aspects of ComIn. <br/>
* To understand how to do this, please refer to the documentations provided in [ComIn Python API](https://icon-comin.gitlab-pages.dkrz.de/comin/d7/de6/md__2builds_2icon-comin_2comin_2doc_2comin__python__api.html) and for more detailed explanation in [Developer documentation](https://icon-comin.gitlab-pages.dkrz.de/comin/#autotoc_md37).
* A member of the ComIn descriptive data structure named <code>decomp_domain</code> equals `0` on prognostic cells.

#### Solution:

```python
@comin.register_callback(comin.EP_ATM_WRITE_OUTPUT_BEFORE)
def simple_python_callbackfct():
    #print("simple_python_callbackfct called!", file=sys.stderr)
    
    jg = 1 # domain 1
    # get the domain descriptive data structure
    domain = comin.descrdata_get_domain(jg)
    # create a mask
    mask = (np.asarray(domain.cells.decomp_domain) == 0)
    # fill the comin_process_id
    comin_process_id_np = np.asarray(comin_process_id)
    comin_process_id_np[mask]  = comin.parallel_get_host_mpi_rank()
    comin_process_id_np[~mask] = -1
```

####  

<div class="alert alert-success">
  <b style="color:#2d4b9b;">Exercise:</b> 
    In the plugin script, <a href="scripts/comin_plugin.py"><code><code style="color:#2d4b9b">scripts/comin_plugin.py</code></a> , within the callback function, you add a print statement that outputs the dimensions of the variable <code>comin_process_id</code>. <br/>
    Re-run the ICON simulation. Can you explain why it has these dimensions?
 
</div>

#### Solution:

```python
@comin.register_callback(comin.EP_ATM_WRITE_OUTPUT_BEFORE)
def simple_python_callbackfct():
    print(np.shape(comin_process_id_np), file=sys.stderr)
```

<figure style="float:right;">
<img src="pic/parallelization-1.png" width="700">
</figure>

* The array's shape will be something like this: (8, 17, 1, 1, 1).
* The first dimension, indicates the number of blocks (`nblk`).
* The second dimension, represents the number of cells within that block (`nproma`).
* The third, fourth and fifth dimensions, are placeholder dimensions.  


* In case of **3D** vaiables the shape would be for example like (8, 25, 17, 1, 1).
* In this case the position 2 is related to the number of vertical levels (`nlev`).


####  

---

## Further Reading and Resources

- [1] ICON Tutorial, Ch. 9: https://www.dwd.de/DE/leistungen/nwv_icon_tutorial/nwv_icon_tutorial.html
- [2] Documentation on the ICON ComIn project website: https://icon-comin.gitlab-pages.dkrz.de/comin/ 
- [3] Python API description for ICON ComIn: https://icon-comin.gitlab-pages.dkrz.de/comin/d7/de6/md__2builds_2icon-comin_2comin_2doc_2comin__python__api.html

---

*Author info: Deutscher Wetterdienst (DWD) 2024 ::  comin@icon-model.org. For a full list of contributors, see CONTRIBUTING in the root directory. License info: see LICENSE file.*